In [2]:
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
llm= ChatGroq(
temperature=0,
api_key="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",
model_name="llama-3.1-70b-versatile"
)

In [4]:
Loader = WebBaseLoader("https://jobs.nike.com/job/R-38999?from=job%20search%20funnel")
page_data = Loader.load().pop().page_content
print(page_data) 


Apply for Software Engineer III

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer IIIBeaverton, OregonBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does more 

In [5]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)
chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)


str

In [6]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res


{'role': 'Software Engineer III',
 'experience': '4+ years of professional experience in application engineering',
 'skills': ['JavaScript (Node.js, React.js, Vue.js, etc.)',
  'Agile processes',
  'Continuous integration',
  'Test automation',
  'AWS CloudFront',
  'AWS S3',
  'AWS Route 53',
  'AWS Certificate Manager',
  'AWS Lambda',
  'GitHub Co-pilot'],
 'description': 'The Digital Employee Experience Software Engineer III will be responsible for design, support, development, and testing of internal employee-facing websites and web applications.'}

In [7]:
type(json_res)


dict

In [8]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df


,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [9]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [10]:
job = json_res
job['skills']

['JavaScript (Node.js, React.js, Vue.js, etc.)',
 'Agile processes',
 'Continuous integration',
 'Test automation',
 'AWS CloudFront',
 'AWS S3',
 'AWS Route 53',
 'AWS Certificate Manager',
 'AWS Lambda',
 'GitHub Co-pilot']

In [11]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/react-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],


In [13]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are lingesh, a business development executive at corer.corer is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of corer 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase corer's portfolio: {link_list}
        Remember you are Lingesh, BDE at corer. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        """
       
)
chain_email = prompt_email | llm 
res = chain_email.invoke(input={"job_description": str(job), "link_list":links})
print(res.content)


Subject: Expert Software Engineering Services for Your Digital Employee Experience

Dear Hiring Manager,

I came across the job description for a Software Engineer III at your esteemed organization, and I'm excited to introduce you to Corer, an AI & Software Consulting company that can help you fulfill your software engineering needs.

With over 4+ years of experience in application engineering, our team of experts specializes in designing, supporting, developing, and testing internal employee-facing websites and web applications. Our expertise in JavaScript (Node.js, React.js, Vue.js, etc.), Agile processes, Continuous integration, Test automation, and AWS services (CloudFront, S3, Route 53, Certificate Manager, and Lambda) aligns perfectly with your requirements.

At Corer, we have a proven track record of delivering high-quality software solutions that meet the needs of our clients. Our portfolio showcases our expertise in full-stack JavaScript development (https://example.com/full-